In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import fnmatch
import tensorflow as tf
from time import sleep
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Dropout,Flatten,BatchNormalization,MaxPooling2D,Activation
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as k
import matplotlib.image as mpimg
import keras

In [2]:
print(os.listdir("/home/suchismita97/malaria streamlit app/archive (2)/cell_images"))


['Parasitized', 'Uninfected', 'cell_images']


In [3]:
imagePatches_0 = glob('/home/suchismita97/malaria streamlit app/archive (2)/cell_images/Uninfected/*.png', recursive=True)
imagePatches_1 = glob('/home/suchismita97/malaria streamlit app/archive (2)/cell_images/Parasitized/*.png', recursive=True)
print(len(imagePatches_0))
print(len(imagePatches_1))

13779
13779


In [4]:
train_para=os.path.join('/home/suchismita97/malaria streamlit app/archive (2)/cell_images/Parasitized/')
train_uninf=os.path.join('/home/suchismita97/malaria streamlit app/archive (2)/cell_images/Uninfected/')
train_para_names=os.listdir(train_para)
train_uninf_names=os.listdir(train_uninf)

nrows=4
ncols=4

pic_index=0

fig=plt.gcf()
fig.set_size_inches(ncols*4,nrows*4)

pic_index+=4

next_para_pix = [os.path.join(train_para,fname) for fname in train_para_names[pic_index-4:pic_index]]
next_uninf_pix = [os.path.join(train_uninf,fname) for fname in train_uninf_names[pic_index-4:pic_index]]

for i, img_path in enumerate(nex t_para_pix+next_uninf_pix):
    sp=plt.subplot(nrows,ncols,i+1)
    sp.axis('Off')
    img=mpimg.imread(img_path)
    plt.imshow(img)
plt.show()

SyntaxError: invalid syntax (<ipython-input-4-4ff166203f14>, line 19)

In [ ]:
x=[]
y=[]
for img in imagePatches_0:
    full_size_image = cv2.imread(img)
    im = cv2.resize(full_size_image, (128, 128), interpolation=cv2.INTER_CUBIC)
    x.append(im)
    y.append(0)
for img in imagePatches_1:
    full_size_image = cv2.imread(img)
    im = cv2.resize(full_size_image, (128, 128), interpolation=cv2.INTER_CUBIC)
    x.append(im)
    y.append(1)
x = np.array(x)
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.2, random_state = 101)
y_train = to_categorical(y_train, num_classes = 2)
y_valid = to_categorical(y_valid, num_classes = 2)

In [ ]:
input_layer = tf.keras.layers.Input([128,128,3])

conv1 = tf.keras.layers.Conv2D(filters = 32 , kernel_size = (5,5) , padding ='Same',
        activation='relu')(input_layer)
pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))( conv1)


conv2 = tf.keras.layers.Conv2D(filters = 64 , kernel_size = (3,3) , padding ='Same',
        activation='relu')(pool1)
pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2,2) , strides=(2,2))(conv2)


conv3 = tf.keras.layers.Conv2D(filters = 128 , kernel_size = (3,3) , padding ='Same',
        activation='relu')(pool2)
pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2,2) , strides=(2,2))(conv3)


conv4 = tf.keras.layers.Conv2D(filters = 128 , kernel_size = (3,3) , padding ='Same',
        activation='relu')(pool3)
pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2,2) , strides=(2,2))(conv4)

flatten = tf.keras.layers.Flatten()(pool4)
dense = tf.keras.layers.Dense(512 , activation = 'relu')(flatten)
out = tf.keras.layers.Dense(2 , activation='sigmoid' )(dense)

model = tf.keras.Model(input_layer , out)


In [11]:
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [12]:
history = model.fit(x_train,y_train,batch_size = 32, epochs = 50, verbose=1,  validation_split=0.2)

Epoch 1/50
552/552 [==============================] - 155s 280ms/step - loss: 2.6391 - accuracy: 0.5579 - val_loss: 0.6415 - val_accuracy: 0.6474
Epoch 2/50
552/552 [==============================] - 152s 276ms/step - loss: 0.6268 - accuracy: 0.6484 - val_loss: 0.6410 - val_accuracy: 0.6415
Epoch 3/50
552/552 [==============================] - 154s 278ms/step - loss: 0.6127 - accuracy: 0.6708 - val_loss: 0.6243 - val_accuracy: 0.6617
Epoch 4/50
552/552 [==============================] - 152s 275ms/step - loss: 0.5893 - accuracy: 0.6928 - val_loss: 0.1990 - val_accuracy: 0.9338
Epoch 5/50
552/552 [==============================] - 152s 276ms/step - loss: 0.1755 - accuracy: 0.9420 - val_loss: 0.1418 - val_accuracy: 0.9574
Epoch 6/50
552/552 [==============================] - 162s 294ms/step - loss: 0.1137 - accuracy: 0.9611 - val_loss: 0.1502 - val_accuracy: 0.9499
Epoch 7/50
552/552 [==============================] - 164s 297ms/step - loss: 0.0981 - accuracy: 0.9655 - val_loss: 0.1430 -

In [29]:

cell_url = 'https://previews.123rf.com/images/someoneice/someoneice1611/someoneice161100105/65539309-normal-red-blood-cells-under-the-microscope.jpg'
cell_path = tf.keras.utils.get_file('cell', origin=cell_url)

img = keras.preprocessing.image.load_img(
    cell_path, target_size=(128, 128)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])


In [33]:
class_names = ['Parasitized', 'Uninfected']


In [34]:
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

This image most likely belongs to Uninfected with a 73.10 percent confidence.


In [35]:
model.save('my_malaria_tf_model')


INFO:tensorflow:Assets written to: my_malaria_tf_model/assets


In [6]:
#reconstructed_model = tf.keras.models.load_model("my_malaria_tf_model")